<a href="https://colab.research.google.com/github/jasgunchandnani/NORMID_DigiDex/blob/main/Model_to_canvas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report

# Load the MNIST dataset and split into training and testing sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data by reshaping the images and normalizing the pixel values to be between 0 and 1
x_train = x_train.reshape((60000, 28, 28, 1)) / 255.0
x_test = x_test.reshape((10000, 28, 28, 1)) / 255.0

# Convert the labels to one-hot encoding
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

# Create the CNN model
model = keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define data augmentation techniques
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10, # Rotate images randomly by up to 10 degrees
    width_shift_range=0.1, # Shift images horizontally by up to 10% of the width
    height_shift_range=0.1, # Shift images vertically by up to 10% of the height
    horizontal_flip=True, # Flip images horizontally
    vertical_flip=False # Don't flip images vertically
)

# Fit the data augmentation generator on the training data
datagen.fit(x_train)

# Train the model on the augmented data
history = model.fit(datagen.flow(x_train, y_train, batch_size=128), epochs=5, validation_data=(x_test, y_test))

# Evaluate the model on the testing data
test_loss, test_accuracy = model.evaluate(x_test, y_test)
y_pred = np.argmax(model.predict(x_test), axis=-1)
y_true = np.argmax(y_test, axis=-1)

# Print the test accuracy
print('Test accuracy:', test_accuracy)

# Compute and print F1 score, recall, and precision
print(classification_report(y_true, y_pred))

# Plot the training and validation accuracy over time
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

# Plot the training and validation loss over time
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

model.save('mnist_cnn_model.h5')


import cv2
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('mnist_cnn_model.h5')

# Create a canvas for the user to draw on
canvas = np.zeros((480, 640), dtype=np.uint8)

# Define a function to handle mouse events
def draw(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(canvas, (x, y), 10, 255, -1)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(canvas, (x, y), 10, 255, -1)

import cv2
import numpy as np
from tensorflow import keras

# Load the trained model
model = keras.models.load_model('mnist_cnn_model.h5')

# Create a canvas for the user to draw on
canvas = np.zeros((480, 640), dtype=np.uint8)

# Define a function to handle mouse events
def draw(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(canvas, (x, y), 10, 255, -1)
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        cv2.circle(canvas, (x, y), 10, 255, -1)

# Create a window to display the canvas
cv2.namedWindow('Canvas')

# Set the mouse callback function for the canvas
cv2.setMouseCallback('Canvas', draw)

while True:
    # Display the canvas
    cv2.imshow('Canvas', canvas)

    # Wait for a key event
    key = cv2.waitKey(1) & 0xFF

    # If the 'c' key is pressed, clear the canvas
    if key == ord('c'):
        canvas[:] = 0

    # If the 'q' key is pressed, quit the program
    elif key == ord('q'):
        break

    # If the 'p' key is pressed, predict the number
    elif key == ord('p'):
        # Preprocess the canvas image and predict the number
        img = cv2.resize(canvas, (28, 28)).reshape(1, 28, 28, 1) / 255.0
        pred = model.predict(img)
        num = np.argmax(pred)

        # Display the predicted number in the console
        print('Predicted number:', num)

        # Clear the canvas
        canvas[:] = 0

# Close the window and release resources
cv2.destroyAllWindows()